In [2]:
import sys
sys.path.append('../')

from preprocessing.imdb_datareader import IMDBDataReader
from preprocessing.smore_datareader import SmoreDataReader
from preprocessing.xgboost_transformer import  XGBoostTransformer
import numpy 

user_path = './ml-100k/u.user'
item_path = './ml-100k/u.item'
user_item = './ml-100k/u.data'
reader = IMDBDataReader()

user_item  = reader.read_user_item_rating(user_item)
users = reader.read_user_data(user_path)
items = reader.read_item_data(item_path)


train_user_item = user_item[:int(len(user_item)*0.8)]
test_user_item = user_item[int(len(user_item)*0.8):]

transformer = XGBoostTransformer(users, items, train_user_item)
smorereader = SmoreDataReader(transformer.u_idx, transformer.i_idx, "rep_dw.txt")
smore_user_vector = smorereader.read_user_data()

total_users = {} 
for k, v in users.items(): 
    if k in smore_user_vector: 
        total_users[k] = v+smore_user_vector[k]

# transformer = XGBoostTransformer(total_users, items, train_user_item)
X_train, Y_train, _, _, nFeatures = transformer.get_feature_vectors(total_users, items, train_user_item)
X_test, Y_test,X_cold_test, Y_cold_test, nFeatures = transformer.get_feature_vectors(total_users, items, test_user_item)

In [3]:
X_train[:10]

[[(0, 21),
  (2, 1),
  (21, 1),
  (24, -0.14095),
  (25, 0.0872827),
  (26, -0.0265255),
  (27, -0.341975),
  (28, -0.00657297),
  (29, -0.197142),
  (30, -0.118619),
  (31, -0.122382),
  (32, 0.155249),
  (33, 0.105875),
  (34, -0.0166278),
  (35, 0.0768601),
  (36, 0.237795),
  (37, -0.146063),
  (38, -0.305308),
  (39, -0.0663939),
  (40, -0.0662637),
  (41, -0.150321),
  (42, 0.00635614),
  (43, -0.0443091),
  (44, 0.188763),
  (45, -0.0199742),
  (46, -0.000867528),
  (47, -0.4852),
  (48, -0.0709278),
  (49, -0.0209386),
  (50, 0.194884),
  (51, 0.108871),
  (52, -0.246217),
  (53, 0.206092),
  (54, -0.138142),
  (55, -0.0414733),
  (56, -0.244284),
  (57, 0.118174),
  (58, 0.215256),
  (59, 0.19001),
  (60, 0.17453),
  (61, 0.0106786),
  (62, -0.10121),
  (63, -0.144248),
  (64, 0.0733318),
  (65, -0.149605),
  (66, 0.244662),
  (67, 0.0827969),
  (68, -0.0678636),
  (69, -0.096278),
  (70, -0.191984),
  (71, -0.186403),
  (72, -0.0442891),
  (73, -0.512105),
  (74, -0.288513),


In [ ]:
!aws s3api create-bucket --bucket recommendation-demo-yianc-0814 --region us-west-2 --create-bucket-configuration LocationConstraint=us-west-2

In [4]:
bucket = 'recommendation-demo-yianc'
prefix = 'sagemaker/xgboost-movielens'
train_key      = 'train'
test_key       = 'test'
test_cold_key  = 'test_cold'


import io,boto3
import sagemaker.amazon.common as smac

def writeToLocalCSV(X, key, Y): 
    f_name = key + ".csv"
    output = open(f_name, 'w')

    for i in range(0, len(X)): 
        line = ''
        raw = X[i]
        line += str(int(Y[i])) 
        line += ' '
        for ele in raw:
            line += str(ele[0])+':'+str(float(ele[1]))+' '
        line += '\n'
        output.write(line)    
    return f_name  


 

train_data = writeToLocalCSV(X_train, train_key, Y_train)    
test_data = writeToLocalCSV(X_test, test_key, Y_test)    
test_cold_data = writeToLocalCSV(X_cold_test, test_cold_key, Y_cold_test)    


In [5]:
!pip install xgboost

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [31]:
import xgboost as xgb
dtrain = xgb.DMatrix('train.csv')
test = xgb.DMatrix('test.csv')
test_cold = xgb.DMatrix('test_cold.csv')



param = {
    'max_depth': 5,  # the maximum depth of each tree
    'eta': 0.3,  # the training step for each iteration
    'alpha':10,
    'silent': 1,  # logging mode - quiet
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'num_class': 2}  # the number of classes that exist in this datset
num_round = 3000  # the number of training iterations
bst = xgb.train(param, dtrain, num_round)


preds = bst.predict(dtrain)


[14:47:14] 80000x108 matrix with 5592968 entries loaded from train.csv
[14:47:14] 2863x108 matrix with 199229 entries loaded from test.csv
[14:47:14] 89x108 matrix with 6243 entries loaded from test_cold.csv
[14:47:14] WARNING: /workspace/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [29]:
from sklearn.metrics import accuracy_score

preds = bst.predict(test)
predictions = []
for i in range(0, len(preds)):
    if preds[i][0] > preds[i][1]:
        predictions.append(0)
    else:
        predictions.append(1)


In [30]:
accuracy = accuracy_score(Y_test, predictions)
accuracy

0.6325532658050995

In [26]:

preds = bst.predict(test_cold)
predictions = []
for i in range(0, len(preds)):
    if preds[i][0] > preds[i][1]:
        predictions.append(0)
    else:
        predictions.append(1)

In [27]:
accuracy = accuracy_score(Y_cold_test, predictions)
accuracy

0.7078651685393258